# Jetpack Compose - StateFlow - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu `StateFlow`.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExcGszZTlwNm5rMDlkdmU3d3FqY21mY2dzMno1cHM5NTM2aHVrcGd5MyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6jHVl7R4F1qkoDXK1R/giphy.gif" width="200" />

`StateFlow` rozszerza funkcjonalność strumieni `Flow` poprzez wprowadzenie pojęcia stanu (*state*). Jest to strumień, który reprezentuje zmieniający się stan i informuje obserwatorów o jego aktualnym stanie.

Główne cechy `StateFlow`:
- Stan: Przechowuje aktualny stan jako niezmienną wartość. Kiedy stan ulega zmianie, nowa wartość jest emitowana do obserwatorów.
- Emitowanie i odbieranie: Emituje wartości stanu i umożliwia ich odbieranie przez obserwatorów. Obserwatorzy są automatycznie informowani o aktualizacjach stanu.

`StateFlow` jest gorącym strumieniem (*hot stream*), co oznacza że emitowanie wartości odbywa się niezależnie od subskrypcji - może emitować wartości nawet wtedy, gdy nie ma aktywnych odbiorców. Nowi odbiorcy, którzy dołączają do strumienia, otrzymują **aktualną wartość stanu** oraz kolejne emitowane wartości.

Jest to zasadnicza różnica między `StateFlow` a `Flow` (cold stream). W przypadku zimnego strumienia, emisja wartości rozpoczyna się dopiero po subskrypcji przez odbiorców.

Nasza aplikacja będzie prostym licznikiem, w którym po przyciśnięciu przycisku zmienimy wartość `StateFlow` i zwiększymy stan licznika.

Rozpocznijmy od zaimplementowania `CounterViewModel`

In [ ]:
class CounterViewModel : ViewModel() {

    private val _stateFlow = MutableStateFlow(0)
    val stateFlow = _stateFlow.asStateFlow()

    fun increase(){
        _stateFlow.value += 1
    }
}

- `private val _stateFlow = MutableStateFlow(0)` - `MutableStateFlow` jest używane do przechowywania i emitowania zmieniającego się stanu - inicjujemy wartością 0
- `val stateFlow = _stateFlow.asStateFlow()` - Tworzy publiczne, tylko do odczytu `StateFlow` poprzez konwersję `MutableStateFlow` na `StateFlow` wykorzystując metofę `asStateFlow()`. `StateFlow` to niemutowalny strumień wartości, który można subskrybować i odczytywać.
- `fun increase() { _stateFlow.value += 1 }` - Zwiększa wartość `_stateFlow.value` o 1. Zmiana wartości `_stateFlow.value` spowoduje automatyczną emisję nowej wartości do subskrybentów.

Zaimplementujmy ui, oraz dodajmy go jako subskrybenta `stateFlow` - implementacja jest identyczna jak w przypadku `Flow`

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            StateFlowBasicsComposeTheme {

                val viewModel: CounterViewModel = viewModel()  // tworzymy instancję viewmodel
                val counter = viewModel.stateFlow.collectAsStateWithLifecycle(0) // tworzymy pole typu State jak w poprzednim przykładzie

                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    Column(
                        modifier = Modifier.fillMaxSize(),
                        verticalArrangement = Arrangement.Center,
                        horizontalAlignment = Alignment.CenterHorizontally
                    ){
                        Text(
                            text = counter.value.toString(), // ustawiamy wartość
                            fontSize = 56.sp,
                            modifier = Modifier.fillMaxWidth(),
                            textAlign = TextAlign.Center
                        )
                        Button(onClick = { viewModel.increase() }) { // Wywołujemy funkcję increase() po naciśnięciu przycisku
                            Text(text = "Increase")
                        }
                    }
                }
            }
        }
    }
}

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExcGszZTlwNm5rMDlkdmU3d3FqY21mY2dzMno1cHM5NTM2aHVrcGd5MyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6jHVl7R4F1qkoDXK1R/giphy.gif" width="200" />

## StateFlow vs Compose State

Nasz `ViewModel` możemy również zaimplementować nieco inaczej.

In [ ]:
class CounterViewModel : ViewModel() {
    private var _composeState by mutableStateOf(0)
    val composeState
        get() = _composeState

    fun increase(){
        _composeState += 1
    }
}

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            StateFlowBasicsComposeTheme {

                val viewModel: CounterViewModel = viewModel()  // tworzymy instancję viewmodel
                val counter = viewModel.composeState

                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    Column(
                        modifier = Modifier.fillMaxSize(),
                        verticalArrangement = Arrangement.Center,
                        horizontalAlignment = Alignment.CenterHorizontally
                    ){
                        Text(
                            text = counter.toString(), // ustawiamy wartość
                            fontSize = 56.sp,
                            modifier = Modifier.fillMaxWidth(),
                            textAlign = TextAlign.Center
                        )
                        Button(onClick = { viewModel.increase() }) { // Wywołujemy funkcję increase() po naciśnięciu przycisku
                            Text(text = "Increase")
                        }
                    }
                }
            }
        }
    }
}

`StateFlow` jest komponentem dostępnym w bibliotece `Kotlin Flow`. Jest to strumień wartości, który reprezentuje zmieniający się stan i informuje obserwatorów o aktualnym stanie. Jest bardziej ogólny i niezależny od biblioteki `Jetpack Compose`. Może być stosowany w dowolnym kontekście, nie tylko w połączeniu z `Jetpack Compose`.

`Compose State` jest natomiast częścią biblioteki `Jetpack Compose`, która dostarcza narzędzia do zarządzania stanem w komponentach interfejsu użytkownika. Jest zoptymalizowany pod kątem integracji z `Jetpack Compose`.

Kilka różnic
- `StateFlow` wpiera operacje transformacji na strumieniach, takich jak `map`, `filter`, `reduce` itp.
- `Flow` jest biblioteką dostępną w Kotlinie, więc klasy korzystające z `Flow`, lub `StateFlow` są łatwiejsze w użyciu. `Compose State` jest ograniczony tylko do biblioteki `Jetpack Compose`, więc jesteśmy ograniczenia do aplikacji i programów wykorzystujących tą bibliotekę - mniejsza przenośność
- `StateFlow` oferuje nieco łatwiejsze zarządzanie *proccess death*

Podsumowując, `StateFlow` jest bardziej ogólnym mechanizmem do zarządzania stanem, który można stosować w różnych kontekstach aplikacji, niezależnie od `Jetpack Compose`. `Compose State` jest specyficznym mechanizmem dostępnym w `Jetpack Compose` do zarządzania stanem w komponentach interfejsu użytkownika,